## 🧩 Aşama 4 — Multi-Threaded Sunucu (Aynı Anda Birden Fazla İstek)
### 💡 Amaç

* Her gelen bağlantı (client) için yeni bir thread başlatmak.
Bu sayede:

>/ isteği çalışırken /sum isteği beklemez.

>Birden fazla kullanıcı aynı anda API’yi kullanabilir.

### ⚙️ Temel Mantık
#### 🔹 Tek iş parçacıklı sistem (şimdiye kadar)
```bash
[Server]
   |
   |---> İstek 1 (işleniyor...)  🕐
   |---> Diğerleri beklemede 😴

🔹 Çok iş parçacıklı sistem (şimdi)
[Server]
   |---> Thread 1 → İstek 1
   |---> Thread 2 → İstek 2
   |---> Thread 3 → İstek 3
   |---> Hepsi paralel 🧠

```
* Python’da bunu threading modülüyle yapacağız.

## 💻 Kod (asama_4.py)

In [1]:
import socket
import threading
import json

In [2]:
HOST ="127.0.0.1"
PORT = 8082
BUF = 4096

def make_response(status,ctype,body):
    status_text = {200:"OK" , 400:"Bad Request" , 404:"Not Found"}.get(status , "OK")
    response = f"""HTTP/1.1{status}{status_text}
Content-Type : {ctype} ; charset=utf-8
Content-Length : {len(body)}
{body}"""

    return response.encode("utf-8")

In [4]:
def parse_request(data):
    headers_part , _ , body = data.partition("\r\n\r\n")
    lines = headers_part.split("\r\n")
    method , part , _ = lines[0].split()

    headers ={}
    for line in lines[1:]:
        if ":" in line:
            key , value = line.split(":" , 1)
            headers[key.strip().lower()] = value.strip
    
    return method , part , headers , body

In [5]:
def handle_get_root():
    body = "<h1>Threaded API 🧠</h1><p>Her istek ayrı bir thread içinde çalışıyor!</p>"
    return 200, "text/html", body

In [7]:
def handle_post_sum(headers, body):
    if headers.get("content-type", "") != "application/json":
        err = json.dumps({"error": "Content-Type application/json olmalı"}, ensure_ascii=False)
        return 400, "application/json", err

    try:
        data = json.loads(body)
    except json.JSONDecodeError:
        err = json.dumps({"error": "Geçersiz JSON"}, ensure_ascii=False)
        return 400, "application/json", err

    if not all(k in data for k in ("a", "b")):
        err = json.dumps({"error": "JSON içinde 'a' ve 'b' anahtarları olmalı"}, ensure_ascii=False)
        return 400, "application/json", err

    try:
        a, b = float(data["a"]), float(data["b"])
        toplam = a + b
    except Exception:
        err = json.dumps({"error": "a ve b sayısal olmalı"}, ensure_ascii=False)
        return 400, "application/json", err

    result = json.dumps({"a": a, "b": b, "sum": toplam, "thread": threading.current_thread().name}, ensure_ascii=False)
    return 200, "application/json", result


In [8]:
ROUTES = {
    ("GET", "/"): handle_get_root,
    ("POST", "/sum"): handle_post_sum,
}

In [9]:
# ---------------------------
# İstekleri işleyen thread fonksiyonu
# ---------------------------
def handle_client(conn, addr):
    print(f"📡 Yeni bağlantı: {addr} | Thread: {threading.current_thread().name}")

    data = b""
    while True:
        chunk = conn.recv(BUF)
        if not chunk:
            break
        data += chunk
        if b"\r\n\r\n" in data:
            break

    raw = data.decode("utf-8", errors="ignore")
    method, path, headers, body = parse_request(raw)

    key = (method, path)
    if key in ROUTES:
        if method == "GET":
            status, ctype, resp_body = ROUTES[key]()
        else:
            status, ctype, resp_body = ROUTES[key](headers, body)
    else:
        status, ctype, resp_body = 404, "text/html", "<h1>404 Not Found</h1>"

    response = make_response(status, ctype, resp_body)
    conn.sendall(response)
    conn.close()
    print(f"✅ {addr} isteği işlendi → {threading.current_thread().name}")

In [ ]:
# ---------------------------
# Sunucu Başlatma (çok iş parçacıklı)
# ---------------------------
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen(5)
    print(f"🚀 Multi-threaded API çalışıyor: http://{HOST}:{PORT}")

    while True:
        conn, addr = srv.accept()
        t = threading.Thread(target=handle_client, args=(conn, addr))
        t.start()

🚀 Multi-threaded API çalışıyor: http://127.0.0.1:8082
📡 Yeni bağlantı: ('127.0.0.1', 62573) | Thread: Thread-394 (handle_client)
✅ ('127.0.0.1', 62573) isteği işlendi → Thread-394 (handle_client)
📡 Yeni bağlantı: ('127.0.0.1', 52562) | Thread: Thread-395 (handle_client)
📡 Yeni bağlantı: ('127.0.0.1', 60233) | Thread: Thread-396 (handle_client)
✅ ('127.0.0.1', 52562) isteği işlendi → Thread-395 (handle_client)
✅ ('127.0.0.1', 60233) isteği işlendi → Thread-396 (handle_client)
📡 Yeni bağlantı: ('127.0.0.1', 50462) | Thread: Thread-397 (handle_client)
✅ ('127.0.0.1', 50462) isteği işlendi → Thread-397 (handle_client)
📡 Yeni bağlantı: ('127.0.0.1', 64076) | Thread: Thread-398 (handle_client)
✅ ('127.0.0.1', 64076) isteği işlendi → Thread-398 (handle_client)
📡 Yeni bağlantı: ('127.0.0.1', 63835) | Thread: Thread-399 (handle_client)
✅ ('127.0.0.1', 63835) isteği işlendi → Thread-399 (handle_client)


## 💡 Ne Öğrendik?

| Özellik                 | Açıklama                                                    |
| ----------------------- | ----------------------------------------------------------- |
| **Threading**           | Her istemci bağlantısı için yeni bir iş parçacığı oluşturur |
| **Paralel işlem**       | Aynı anda birden fazla istek işlenir                        |
| **Gelişmiş performans** | I/O beklemeleri sırasında diğer istekler çalışır            |
| **Thread ismi**         | `threading.current_thread().name` ile log’a yazılır         |
| **Thread güvenliği**    | Şu an için veri paylaşımı yok, bu yüzden güvenli            |
